# **ARP Spoof Detection - ML MODEL**

## Reading the CSV file and installing importing dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, TimeDistributed, Flatten

In [ ]:
import tensorflow
from tensorflow.keras.layers import InputLayer, GRU, BatchNormalization, Conv1D, MaxPooling1D, Bidirectional, Dense, Flatten, PReLU, LSTM, ReLU, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import keras

In [40]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/arpcode.csv')

In [ ]:
print(df.head())

       Time                 Source        Destination Protocol  Length  Info
0  0.000000  PCSSystemtec_3a:59:87  7e:57:ab:7a:2b:f5      ARP      42     0
1  2.001191  PCSSystemtec_3a:59:87  7e:57:ab:7a:2b:f5      ARP      42     0
2  4.002254  PCSSystemtec_3a:59:87  7e:57:ab:7a:2b:f5      ARP      42     0
3  6.002604  PCSSystemtec_3a:59:87  7e:57:ab:7a:2b:f5      ARP      42     0
4  8.003296  PCSSystemtec_3a:59:87  7e:57:ab:7a:2b:f5      ARP      42     0


## Making the Model (Wrapping CNN in Time Distributed Layers using CNN-LSTM Architecture)

In [41]:
sequence_length = 10
window_size = 1
feature_count = 3
lrcn_model = Sequential()

# first convolutional block
lrcn_model.add(InputLayer(input_shape=(window_size, feature_count)))
lrcn_model.add(Conv1D(filters=32, kernel_size=3, padding='causal', activation='ReLU'))
lrcn_model.add(LeakyReLU())
lrcn_model.add(MaxPooling1D(pool_size=1))
lrcn_model.add(BatchNormalization())

#Adding LSTM layers
lrcn_model.add(LSTM(30, activation='relu', dropout=0.3))
lrcn_model.add(BatchNormalization())

# output dense layer
lrcn_model.add(Dense(1, activation='sigmoid'))
lrcn_model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])
lrcn_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_10 (Conv1D)          (None, 1, 32)             320       
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 1, 32)             0         
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 1, 32)             0         
 g1D)                                                            
                                                                 
 batch_normalization_12 (Ba  (None, 1, 32)             128       
 tchNormalization)                                               
                                                                 
 lstm_6 (LSTM)               (None, 30)                7560      
                                                                 
 batch_normalization_13 (Ba  (None, 30)              

## Splitting the Dataset into Testing and Training

In [46]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')

df['HourOfDay'] = df['Time'].dt.hour + df['Time'].dt.minute / 60 + df['Time'].dt.second / 3600

df['TimeSin'] = np.sin(2 * np.pi * df['HourOfDay']/24.0)
df['TimeCos'] = np.cos(2 * np.pi * df['HourOfDay']/24.0)

y = df['Info']
X = df[['Length', 'TimeSin', 'TimeCos']]

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape((2528,1,3))
X_test = X_test.reshape((632, 1, 3))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2528, 1, 3) (632, 1, 3) (2528,) (632,)


## Training and Testing the data

In [47]:
history = lrcn_model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/200
40/40 [==============================] - 4s 22ms/step - loss: 0.7651 - accuracy: 0.5103 - val_loss: 0.6671 - val_accuracy: 0.9304
Epoch 2/200
40/40 [==============================] - 0s 7ms/step - loss: 0.6889 - accuracy: 0.5886 - val_loss: 0.6440 - val_accuracy: 0.9304
Epoch 3/200
40/40 [==============================] - 0s 8ms/step - loss: 0.6718 - accuracy: 0.6388 - val_loss: 0.6250 - val_accuracy: 0.9304
Epoch 4/200
40/40 [==============================] - 0s 9ms/step - loss: 0.6460 - accuracy: 0.6808 - val_loss: 0.6076 - val_accuracy: 0.9304
Epoch 5/200
40/40 [==============================] - 0s 8ms/step - loss: 0.6401 - accuracy: 0.7037 - val_loss: 0.5912 - val_accuracy: 0.9304
Epoch 6/200
40/40 [==============================] - 0s 6ms/step - loss: 0.6270 - accuracy: 0.7148 - val_loss: 0.5752 - val_accuracy: 0.9304
Epoch 7/200
40/40 [==============================] - 0s 5ms/step - loss: 0.6130 - accuracy: 0.7286 - val_loss: 0.5609 - val_accuracy: 0.9304
Epoch 8/200


In [ ]:
model_preds = lrcn_model(X_test)

In [45]:
len(model_preds), len(y_test)

(632, 632)

In [49]:
for x, y in zip(model_preds, y_test):
  if y==1:
    print(x, y)

tf.Tensor([[0.28170305]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.2264389]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.29738575]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.11445358]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.2858905]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.2914685]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.28292933]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.27509218]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.23596704]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.30708066]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.20760183]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.28070927]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.18611397]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.27509218]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.30373457]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.29308727]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.15655237]], shape=(1, 1), dtype=float32) 1
tf.Tensor([[0.25217062]], shape=(1

In [ ]:
a = df[['Length', 'TimeSin', 'TimeCos', 'Info']]
a['Info'].value_counts()

0    2894
1     266
Name: Info, dtype: int64

## Result

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators=100, random_state=42)

X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

clf.fit(X_train_flat, y_train)
y_pred = clf.predict(X_test_flat)

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:",100*accuracy,"%")


Accuracy: 91.93037974683544 %
